In [1]:
import torchaudio
import os
# import wget
import scipy.io

## Get Audio Data From Online Drive

In [ ]:
Dateset_URL = ''
HOME = 'C:\Users\claud\OneDrive\Documents\School\T7\50.039 DL\Deep_Learning_Denoiser\noisy_samples\noise_sample_177659.wav'

dataset = wget.download(URL)

In [12]:
# path = '.\\noisy_samples\\common_voice_en_1.mp3'
path = '.\\data\\clean_audio_WAV\\common_voice_en_1.wav'
# path = '.\\noisy_samples\\noise_sample_62094.wav'
waveform, sample_rate = torchaudio.load(path, normalize=True)

In [14]:
waveform

tensor([[0., 0., 0.,  ..., 0., 0., 0.]])

In [5]:
waveform.shape

torch.Size([1, 107136])

In [29]:
dir = 'E:\\Dev\\ML\\audio_denoiser\\clean_audio'

for filename in os.listdir(dir):
    path = os.path.join(dir, filename)
    rate, data = scipy.io.wavfile.read(path)
    print(data)

ValueError: File format b'ID3\x04' not understood. Only 'RIFF' and 'RIFX' supported.